# **音乐节拍实时分析（Track the Beat on the Fly)**

## **Demo 视频**

播放器播放音乐，基于 **[madmom](https://github.com/CPJKU/madmom)** 库使用 Python 实时分析音乐的节拍，灯光与 Scratch 彩虹按钮同步响应，Python 与 Scratch 的联系通过 Adapter 实现。

In [1]:
import IPython.display as ipd

ipd.Video("video/demo1_tracktheBeat.mp4", width=800, height=600)

&emsp;

## **Demo 代码**

**视频中的示例项目主要包括三个部分：**

1. 在 JupyterLab 内部使用 Python 实时分析音乐的节拍；

2. 使用 Python，通过 WS2812 灯带的 WIFI 控制器，对灯光编程控制，使其响应音乐节拍闪烁

3. 使用 Scratch 制作彩虹按钮，项目[在此](https://create.codelab.club/projects/11063/editor/)

Python 与 Scratch 的沟通通过 [CodeLab Adapter](https://adapter.codelab.club/get_start/gs_install/) EIM 插件实现

&emsp;

**需要留意的问题：**
  
+ 音乐节拍分析使用的音源
    
    当我们播放音乐时，电脑 **同时** 在对所播放的音乐做实时 **录音**，而我们分析音乐节拍的音源就是麦克风实时 **录下的音乐**；
    
    我们可能会直接使用电脑的内置音箱播放音乐，也可能会外接耳机或是音箱播放音乐，它们也都有自己内置的麦克风可以录音，所以这时候，要注意选择外接耳机或是音箱的麦克风作为音乐节拍分析的音源；
        
    Linux 系统（包括树莓派）可以安装使用 PulseAudio Volume Control 来查看选择音源
    
+ Pyhon 代码中 EIM NODE_ID 与配套 Scratch 项目内 EIM 积木中输入的 NODE_ID 名称必须一致
    
+ CodeLab Adapter 是否连接正常并已开启运行

如果运行代码没有报错但是 Scratch 彩虹按钮没有闪动或是灯光没有响应，很可能是因为以上这几个问题造成的。

In [ ]:
import numpy as np
import time
import random 
#import rpi_ws281x
#from rpi_ws281x import PixelStrip, Color            # 树莓派驱动灯带依赖的 python 库
from madmom.features.beats import DBNBeatTrackingProcessor, RNNBeatProcessor             # 音乐节拍分析依赖的 python 库
from madmom.models import BEATS_LSTM
from madmom.processors import IOProcessor, process_online
from codelab_adapter_client import AdapterNode                       # 沟通 python 与 Scratch 代码依赖的 Adapter



# 在 Adapter EIM node 中定义接收和发送消息的函数
class MyNode(AdapterNode):
    NODE_ID = "eim/tracktheBeat"     # 这个 ID 名（即 "eim/"后面的部分自己定义，配套 Scratch 项目中使用的接收和发送 EIM 消息的积木内要写与此一致的名字
    
    def __init__(self):
        super().__init__()
        self.is_ready = False
        
    def send_data(self, content):    # 向 Scratch 发送消息用
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)

        
node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)


"""
# 下面是基于 rpi_wx281x 库设置的灯带基本参数，参考库内用例
LED_COUNT = 30              # 灯珠数量
LED_PIN = 18                # 灯带写入数据线连接的树莓派 pin：GPIO 18
LED_FREQ_HZ = 800000        # 默认参数
LED_DMA = 10                # 默认参数
LED_BRIGHTNESS = 200        # 灯珠的亮度，范围 0-255，自己根据需要修改
LED_INVERT = False          # 默认参数
LED_CHANNEL = 0             # 默认参数



# 根据以上参数初始化灯带
strip = PixelStrip(LED_COUNT, LED_PIN, LED_FREQ_HZ, LED_DMA, LED_INVERT, LED_BRIGHTNESS, LED_CHANNEL)
strip.begin()
"""



# 利用 madmom 实时分析音乐节拍所需的参数
kwargs = dict(
    online = True,
    fps = 100,
    infile = None,
    outfile = None, 
    nn_files = None
)



# 对应 Scratch 项目中 8 种彩虹按钮，预定义 8 种颜色
color_list = ["red", "orange", "yellow", "green", "cyan", "blue", "purple", "violet"]


"""
# 灯带颜色效果函数定义，很简单，就是让所有灯珠呈现某种颜色
def ColorWipe(strip, color):
    for i in range(strip.numPixels()):
        strip.setPixelColor(i, color)
    strip.show()  
"""    
    
    
# 每当节拍被识别到后，就会调用下面这个函数
# 会随机从 8 种颜色中选取一种，使灯光渲染颜色，Scratch 中对应颜色按钮闪动
def beat_callback(beats, output=None):
    if len(beats) > 0:
        color = random.choice(color_list)
        #print(color)
        if color == 'red':
            node.send_data("red")
            #ColorWipe(strip, Color(255, 0, 0))
        if color == 'blue':
            node.send_data("blue")
            #ColorWipe(strip, Color(0, 0, 255))
        if color == 'green':
            node.send_data("green")
            #ColorWipe(strip, Color(0, 255, 0))
            
        if color == 'purple':
            node.send_data("purple")
            #ColorWipe(strip, Color(255, 0, 255))
            
        if color == 'yellow':
            node.send_data("yellow")
            #ColorWipe(strip, Color(255, 255, 0))
            
        if color == 'cyan':
            node.send_data("cyan")
            #ColorWipe(strip, Color(0, 180, 100))
            
        if color == 'violet':
            node.send_data("violet")
            #ColorWipe(strip, Color(187, 51, 133))
            
        if color == 'orange':
            node.send_data("orange")
            #ColorWipe(strip, Color(255, 85, 0))
            
 
  
# madmom 实时分析音乐节拍用到的 processor            
    
in_processor = RNNBeatProcessor(**kwargs)
beat_processor = DBNBeatTrackingProcessor(**kwargs)
out_processor = [beat_processor, beat_callback]
processor = IOProcessor(in_processor, out_processor)
process_online(processor, **kwargs)

&emsp;
&emsp;

## **音乐节拍实时分析代码**

[**Madmom**](https://madmom.readthedocs.io/en/latest/modules/features/beats.html) 是一个非常好的 Python 库，可以用来提取音频信号的底层特征（feature extraction），基于这些特征做进一步的分析（feature analysis），从而获得更高水平上音乐相关的特性，在这里我们关注的是音乐的节拍信息。

关于 Madmom，有如下几个特点值得简单说明一下，更详细的介绍可以阅读仓库内的[官方文献](paper/madmom.pdf)以及[线上文档](https://madmom.readthedocs.io/en/latest/modules/features/beats.html)。

+ 面向对象设计，主要有两个类，一个是 Data，可以看作 Numpy ndarray 的亚类（subclass），用来存放信号数据；另一个是 Processor，用来定义如何转化、处理数据，可以利用系统多核 CPU 并行分析处理数据

+ 采用机器学习、特别是神经网络相关的先进算法，提取并分析音频特征

+ 库内 `bin` 文件夹下有多种可直接运行的程序

+ 依赖很少


In [ ]:
from madmom.features.beats import DBNBeatTrackingProcessor, RNNBeatProcessor
from madmom.models import BEATS_LSTM
from madmom.processors import IOProcessor, process_online

kwargs = dict(
    online = True,
    fps = 100,
    correct = True,
    infile = None,
    outfile = None,
    max_bpm = 170,
    min_bpm = 100,
    #nn_files = [BEATS_LSTM[0], BEATS_LSTM[1], BEATS_LSTM[2]],
    num_frames = 1,
    origin = 'stream'
)

def beat_callback(beats, output=None):
    if len(beats) > 0:
        # Do something with the beat (for now, just print the array to stdout)
        print(beats)

        

    
in_processor = RNNBeatProcessor(**kwargs)
beat_processor = DBNBeatTrackingProcessor(**kwargs)
out_processor = [beat_processor, beat_callback]
processor = IOProcessor(in_processor, out_processor)
process_online(processor, **kwargs)
    
